# Power Co  Churn Analysis

  Powerco is a major gas and electricity utility that supplies corporate, SME (Small & Medium enterprises), and residential customers. The power liberalization of the energy market in Europe has led to significant customer churn, especially in the SME segment. They have partnered with BCG to help diagnose the source of churning SME customers.

  A fair hypothesis is that price changes affect customer churn. Therefore, it is helpful to know which customers are more (or less) likely to churn at their current price, for which a good **predictive model** could be useful.
  
## Hypothesis Framing

* **Null Hypothesis** : The change in customer price affects customer churn.
* **Alternative Hypothesis** : The change in customer price does not affect customer churn.

## Understanding PowerCo Pricing

The PowerCo company comes under the category of the utility industry. Generally, Utility Industry follows a dynamic pricing method to charge their service. **Dynamic Pricing** or surge pricing is a pricing strategy in which businesses set flexible prices for products or services based on current market demands. There were several existing algorithms to calculate the price dynamically.

## Electricity Pricing

**Fixed Charges:**
These charges are electricity costs that don’t change no matter how much electricity you use. Fixed charges are designed to recover the basic cost of electric service. The fixed demand charges can be charged as per the use of utility service in the area and can result in higher charges due to the higher use of the utility.

**Peak Pricing or Tariff Pricing:** 
Such a type of tariff is similar to peak load tariffs. The only difference is that the seasonal tariff measures the peak hour of the year and the peak tariff calculates it for the day.  If the power consumption is high, then it is known as the on-peak tariff, and for low power consumption, it is called the off-peak load tariff.




In [ ]:
import numpy as np #Numerical Analysis
import pandas as pd #ETL
import matplotlib.pyplot as plt #Data Visualization
import seaborn as sns #Data Visualization
import warnings #Supressing Unnecessary Warnings

In [ ]:
#Options and Settings
warnings.simplefilter('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
colors = sns.color_palette("Set3", 10)

In [ ]:
#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score,classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler,QuantileTransformer
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

## Importing Dataset
#### Data Description

**client_data.csv**

* id = client company identifier
* activity_new = category of the company’s activity
* channel_sales = code of the sales channel
* cons_12m = electricity consumption of the past 12 months
* cons_gas_12m = gas consumption of the past 12 months
* cons_last_month = electricity consumption of the last month
* date_activ = date of activation of the contract
* date_end = registered date of the end of the contract
* date_modif_prod = date of the last modification of the product
* date_renewal = date of the next contract renewal
* forecast_cons_12m = forecasted electricity consumption for next 12 months
* forecast_cons_year = forecasted electricity consumption for the next calendar year
* forecast_discount_energy = forecasted value of current discount
* forecast_meter_rent_12m = forecasted bill of meter rental for the next 2 months
* forecast_price_energy_off_peak = forecasted energy price for 1st period (off peak)
* forecast_price_energy_peak = forecasted energy price for 2nd period (peak)
* forecast_price_pow_off_peak = forecasted power price for 1st period (off peak)
* has_gas = indicated if client is also a gas client
* imp_cons = current paid consumption
* margin_gross_pow_ele = gross margin on power subscription
* margin_net_pow_ele = net margin on power subscription
* nb_prod_act = number of active products and services
* net_margin = total net margin
* num_years_antig = antiquity of the client (in number of years)
* origin_up = code of the electricity campaign the customer first subscribed to
* pow_max = subscribed power
* churn = has the client churned over the next 3 months

**price_data.csv**

* id = client company identifier
* price_date = reference date
* price_off_peak_var = price of energy for the 1st period (off peak)
* price_peak_var = price of energy for the 2nd period (peak)
* price_mid_peak_var = price of energy for the 3rd period (mid peak)
* price_off_peak_fix = price of power for the 1st period (off peak)
* price_peak_fix = price of power for the 2nd period (peak)
* price_mid_peak_fix = price of power for the 3rd period (mid peak)


In [ ]:
price_data = pd.read_csv('../input/powerco/price_data.csv')
client_data = pd.read_csv('../input/powerco/client_data.csv')

## User Defined Functions

In [ ]:
# Function to label partitions of pie chart
def annotation_labeling(lbl, vals):
    absolute = int(np.round(lbl/100.*np.sum(vals)))
    return "{:.1f}%\n({:d})".format(lbl, absolute)

In [ ]:
# To find the upper bound and lower bound of the distribution using inter-quantile range
def iqr_range(arr):
    q1 = np.quantile(arr,0.25)
    q3 = np.quantile(arr,0.75)
    iqr = q3-q1
    return q1-(iqr*1.5),q3+(iqr*1.5)

In [ ]:
def outlier_removal(df,column,drop_percentage=0.05):
    
    actual_rows = df.shape[0]
    lower_bound,upper_bound = iqr_range(df[column].values)
    outliers = df[ (df[column]<lower_bound) | (df[column]>upper_bound) ]
    print('Total No of rows in actual dataframe : ',actual_rows,'\nTotal No of rows in outliers : ',outliers.shape[0])
    if outliers.shape[0] <= round(actual_rows*drop_percentage):
        df.drop(outliers.index,inplace=True)
    
    return df

## Exploring Price Data

In [ ]:
price_data.head()

In [ ]:
price_data.info()

In [ ]:
#Checking for duplicates
price_data[price_data.duplicated()]

In [ ]:
#Changing datatype : price date => object -> datetime64
price_data = price_data.astype({'price_date' : 'datetime64'})

In [ ]:
print('No of unique clients : ',price_data.id.nunique())
#basic Stats
price_data.describe()

## Merging Client Dataset and Price Dataset
Merging price and client dataset using id

In [ ]:
client_churn_info = client_data[['id','churn']]
price_df = client_churn_info.merge(price_data,on='id')

In [ ]:
#Merged Dataset
price_df.head()

## Univariant Analysis
Plotting histogram to see the distribution of the data.

In [ ]:
price_df.groupby(['id','price_date']).mean().hist(figsize=(20,10))
plt.show()

*The HistPlot shows that the data is skewed and show be treated before model creation.*

## Plotting Energy and Power Prices

In [ ]:
#Spliting the dataset into chrun and non-churn price by price date
churn_grp_price = price_df[price_df['churn']==1].groupby(['price_date']).mean()
non_churn_grp_price = price_df[price_df['churn']==0].groupby('price_date').mean()

In [ ]:
#Plotting average price of energy by month 
plt.figure(figsize=(15,3))
plt.xticks(rotation=45)
plt.subplot(131)
non_churn_grp_price.price_off_peak_var.plot()
churn_grp_price.price_off_peak_var.plot()
plt.xticks(rotation=45)
plt.legend(['Not Churn','Churn'])
plt.title('Energy price at off peak')
plt.subplot(132)
non_churn_grp_price.price_peak_var.plot()
churn_grp_price.price_peak_var.plot()
plt.legend(['Not Churn','Churn'])
plt.title('Energy price at peak')
plt.xticks(rotation=45)
plt.subplot(133)
non_churn_grp_price.price_mid_peak_var.plot()
churn_grp_price.price_mid_peak_var.plot()
plt.legend(['Not Churn','Churn'])
plt.title('Energy price at mid peak')
plt.xticks(rotation=45)
plt.suptitle('Energy price of Non-Churn vs Churn Customers')
plt.subplots_adjust(top=0.8)
plt.show()

In [ ]:
#Plotting average price of power by month
plt.figure(figsize=(15,3))
plt.xticks(rotation=45)
plt.subplot(131)
non_churn_grp_price.price_off_peak_fix.plot()
churn_grp_price.price_off_peak_fix.plot()
plt.xticks(rotation=45)
plt.legend(['Not Churn','Churn'])
plt.title('Power price at off peak')
plt.subplot(132)
non_churn_grp_price.price_peak_fix.plot()
churn_grp_price.price_peak_fix.plot()
plt.legend(['Not Churn','Churn'])
plt.title('Power price at peak')
plt.xticks(rotation=45)
plt.subplot(133)
non_churn_grp_price.price_mid_peak_fix.plot()
churn_grp_price.price_mid_peak_fix.plot()
plt.legend(['Not Churn','Churn'])
plt.title('Power price at mid peak')
plt.xticks(rotation=45)
plt.suptitle('Power price of Non-Churn vs Churn Customers')
plt.subplots_adjust(top=0.8)
plt.show()

There is some variation of price between churn and un-churn clients. Churned clients have slightly low off-peak energy prices and high off-peak prices. But we can't confidently say this is the factor for churn. We need to analyze the data further to arrive at a concrete conclusion.

## Correlational Heatmap
Before merging the dataset with the client dataset, we should conduct a correlation analysis to identify the features replicating the same information.

In [ ]:
plt.figure(figsize=(5,5))
pd_corr = price_data.corr()
mask = np.triu(np.ones_like(pd_corr))
sns.heatmap(pd_corr,annot=True,cmap="crest",linewidth=.5,mask=mask)
plt.title('Correlation Plot')
plt.show()

In [ ]:
price_data.drop(['price_peak_var','price_peak_fix','price_mid_peak_var'],axis=1,inplace=True)

*High co-relation coefficient values between two features denote the high dependency between them. It shows that the two features convey the same information. So one of the features can be dropped.*

## Feature Engineering
Since the price information is monthly, we can't able to merge it directly with the client data. Some feature transformations need to be done to derive the yearly prices. The off-peak and mid-peak of energy follow a linear flow, so we can take the average of the monthly prices. The energy off-peak price was high at the beginning of the year and low at the end of the year. So we take the price difference as a consolidated value.

In [ ]:
#Filtering out the January and December energy off peak price
price_off_peak_energy = price_data[['id','price_off_peak_var']]
jan_prices = price_off_peak_energy.groupby('id').price_off_peak_var.first().reset_index().rename(columns={'price_off_peak_var':'price_off_peak_var_jan'})
dec_prices = price_off_peak_energy.groupby('id').last().price_off_peak_var.reset_index().rename(columns={'price_off_peak_var':'price_off_peak_var_dec'})

In [ ]:
price_data.drop('price_off_peak_var',axis=1,inplace=True)
#Taking average of Power off-peak and mid-peak
price_data = price_data.groupby('id').mean().reset_index()

In [ ]:
#Calculating Energy off peak difference
price_data = price_data.merge(jan_prices,on='id').merge(dec_prices,on='id')
price_data['energy_off_peak_variation'] = price_data.price_off_peak_var_jan - price_data.price_off_peak_var_dec
price_data.drop(['price_off_peak_var_jan','price_off_peak_var_dec'],axis=1,inplace=True)

In [ ]:
#Final price dataset
price_data.head()

## Exploring Client Data

In [ ]:
#Inspecting Client Data
client_data.head()

**The *activity_new* (category of the company's activity) data is not provided.**

In [ ]:
client_data.info()

In [ ]:
#Checking for duplicates
client_data[client_data.duplicated()]

In [ ]:
#Changing datatype
client_data = client_data.astype({
    'date_activ' : 'datetime64',
    'date_end' : 'datetime64',
    'date_modif_prod' : 'datetime64',
    'date_renewal' : 'datetime64',
})

### Data Cleaning and Feature Engineering

In [ ]:
#Changing hashed values to meaningful labels for easy understanding.
print('Unique Sales Channels : \n',client_data.channel_sales.unique())
print('\nUnique Origin Campaign : \n',client_data.origin_up.unique())

In [ ]:
channel_mask = {
    'MISSING':'missing_data',
    'foosdfpfkusacimwkcsosbicdxkicaua':'channel_1',
    'lmkebamcaaclubfxadlmueccxoimlema':'channel_2',
    'usilxuppasemubllopkaafesmlibmsdf':'channel_3',
    'ewpakwlliwisiwduibdlfmalxowmwpci':'channel_4',
    'epumfxlbckeskwekxbiuasklxalciiuu':'channel_5',
    'sddiedcslfslkckwlfkdpoeeailfpeds':'channel_6',
    'fixdbufsefwooaasfcxdxadsiekoceaa':'channel_7',
}
origin_mask = {
    'lxidpiddsbxsbosboudacockeimpuepw' : 'origin_1',
    'kamkkxfxxuwbdslkwifmmcsiusiuosws' : 'origin_2',
    'ldkssxwpmemidmecebumciepifcamkci' : 'origin_3',
    'usapbepcfoloekilkwsdiboslwaxobdp' : 'origin_4',
    'ewxeelcelemmiwuafmddpobolfuxioce' : 'origin_5',
    'MISSING' : 'origin_missing'
}
client_data.replace({
    'has_gas' : {
        't':1,'f':0
    },
    'channel_sales':channel_mask,
    'origin_up':origin_mask,
},inplace=True)
#Labeled Data
client_data.head()

In [ ]:
#contract_modification_interval : no of years in which the client last modified their contract.
#contract_interval : no of years client travelled with us.
client_data['contract_modification_interval'] = client_data.date_modif_prod.dt.year - client_data.date_activ.dt.year
client_data['contract_interval'] = client_data.date_end.dt.year - client_data.date_activ.dt.year

### Merging Client and Price Data

In [ ]:
df = client_data.merge(price_data,on='id')
print('Total No of Clients in Price Dataset : ',price_data.id.nunique(),'\nTotal No of Clients in Client Dataset : ',client_data.id.nunique(),'\nTotal No of Clients after merging : ',df.id.nunique())

**There is no client data for 1490 clients.**

In [ ]:
df.head()

In [ ]:
df.info()

### Correlation Analysis
We have around 31 features.Some of features are derived quantities, by conducting correlation analysis we can able to reduce the highly correlated features.

In [ ]:
plt.figure(figsize=(15,15))
df_corr = df.corr()
mask = np.triu(np.ones_like(df_corr))
sns.heatmap(df_corr,annot=True,cmap="crest",linewidth=.5,mask=mask)
plt.title('Correlation Plot')
plt.show()

### Dropping Highly Correlated and Unwanted Columns

In [ ]:
df.drop([
    #Highly correlated columns
    'cons_last_month','forecast_cons_12m', 'forecast_cons_year','forecast_meter_rent_12m',
    'forecast_meter_rent_12m', 'forecast_price_energy_off_peak','forecast_price_energy_peak', 
    'forecast_price_pow_off_peak','margin_gross_pow_ele', 'nb_prod_act',
    'num_years_antig',
    #Unwanted Columns
    'id','date_activ','date_end','date_renewal','date_modif_prod',
],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
#Duplicate Records
df[df.duplicated(subset=['cons_12m',
       'imp_cons', 'net_margin',
       'pow_max', 'churn', 'price_off_peak_fix', 'price_mid_peak_fix',
       'energy_off_peak_variation'])]

In [ ]:
df.drop_duplicates(subset=['cons_12m',
       'imp_cons', 'net_margin',
       'pow_max', 'churn', 'price_off_peak_fix', 'price_mid_peak_fix',
       'energy_off_peak_variation'],ignore_index=True,inplace=True)

In [ ]:
df.columns

In [ ]:
#univariant Analysis
df.hist(figsize=(15,10))
plt.show()

From the histograms, we can say that there is outliers and sknewness in the continous value features needed.

In [ ]:
#Removing outliers if the feature's outliers count is less than a certain threshold from the total dataset.
#Note: The threshold of the outlier removal was selected in the manner that the minority classes were preserved
for col in ['cons_12m', 'cons_gas_12m', 'imp_cons', 'margin_net_pow_ele', 'net_margin', 
       'pow_max',  'price_mid_peak_fix','energy_off_peak_variation','price_off_peak_fix']:
    df = outlier_removal(df,col,0.08)
    print(df.churn.value_counts())

In [ ]:
#Feature Distribution after removing outliers
ax = df.hist(figsize=(15,10),column=['cons_12m', 'cons_gas_12m', 'imp_cons', 'margin_net_pow_ele', 'net_margin', 
       'pow_max',  'price_mid_peak_fix','energy_off_peak_variation','price_off_peak_fix'],bins=10)
plt.show()

## Feature Analysis & Visualizations
### Client Churn

In [ ]:
plt.pie(df.churn.value_counts(),
        labels=['Non Churn','Churn'], 
        autopct=lambda x : annotation_labeling(x,df.churn.value_counts().values))
plt.title('Churn VS Non-Churn Clients')
plt.show()

In [ ]:
#Spliting the dataset into chrun and non-churn price by price date
churn_df = df[df['churn']==1]
non_churn_df = df[df['churn']==0]

### Discount

In [ ]:
'''
    Here we're making assumption that customers with discount will have the forecasted discount.
    By that we divide clients as client with discount and without discount
'''
abs_values_d = df[df.forecast_discount_energy != 0].churn.value_counts(ascending=False)
rel_values_d = df[df.forecast_discount_energy != 0].churn.value_counts(ascending=False, normalize=True).values * 100
dlbls = [f'{p[0]} ({p[1]:.0f}%)' for p in zip(abs_values_d, rel_values_d)]

abs_values_wd = df[df.forecast_discount_energy == 0].churn.value_counts(ascending=False)
rel_values_wd = df[df.forecast_discount_energy == 0].churn.value_counts(ascending=False, normalize=True).values * 100
wdlbls = [f'{p[0]} ({p[1]:.0f}%)' for p in zip(abs_values_wd, rel_values_wd)]

plt.figure(figsize=(15,5))
plt.subplot(121)
ax1 = sns.countplot(df[df.forecast_discount_energy != 0].churn)
ax1.bar_label(container=ax1.containers[0], labels=wdlbls)
plt.title('Clients with discount')
plt.subplot(122)
ax2 = sns.countplot(df[df.forecast_discount_energy == 0].churn)
ax2.bar_label(container=ax2.containers[0], labels=dlbls)
plt.title('Clients without discount')
plt.suptitle('')
plt.show()

### Contract Interval (Client's antiques)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(122)
plt.title('Churn Client')
sns.countplot(churn_df.contract_interval,palette=['#42bff5'])
plt.subplot(121)
sns.countplot(non_churn_df.contract_interval,palette=['#42bff5'])
plt.title('Non Churn Client')
plt.suptitle('Contract Interval')
plt.show()

### Contract Modification

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(122)
plt.title('Churn Client')
sns.countplot(churn_df.contract_modification_interval,palette=['#42f581'])
plt.subplot(121)
sns.countplot(non_churn_df.contract_modification_interval,palette=['#42f581'])
plt.title('Non Churn Client')
plt.suptitle('Contract Interval')
plt.show()

### Origin Campaign

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(122)
plt.title('Churn Client')

_,_,lbl_text = plt.pie(non_churn_df.origin_up.value_counts(),
        labels=non_churn_df.origin_up.value_counts().index,
        autopct=lambda x : annotation_labeling(x,non_churn_df.origin_up.value_counts()),colors=colors)

plt.subplot(121)
plt.title('Non Churn Client')
_,_,lbl_text = plt.pie(churn_df.origin_up.value_counts(),
        labels=churn_df.origin_up.value_counts().index,
        autopct=lambda x : annotation_labeling(x,churn_df.origin_up.value_counts()),colors=colors)
plt.suptitle("Origin Campaigns")
plt.show()

### Sales Channel

In [ ]:
clients_channel_count = df['channel_sales'].value_counts()
data = {
    'Channel Name' : clients_channel_count.index.values,
    'Channel Count' : clients_channel_count.values,
    'Percentage' : (clients_channel_count.values/client_data.shape[0])*100
}
channel_sales_data = pd.DataFrame(data)

In [ ]:
churn_channel_sales =churn_df['channel_sales'].value_counts()
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10, 5))
plt.figure(figsize=(10,5))
#To avoid clumsiness in chart, last three rows are not taken into account for the plotting.
_,_,ax1_text = ax1.pie(x=channel_sales_data['Channel Count'][:-3],
                       labels=channel_sales_data['Channel Name'][:-3],
                       autopct = lambda x : annotation_labeling(x,channel_sales_data['Channel Count'].values),
                       colors=colors)
ax1.set_title('Total Channel Sales Count')
_,_,ax2_text=ax2.pie(x=churn_channel_sales.values,labels=churn_channel_sales.index,autopct = lambda x : annotation_labeling(x,churn_channel_sales.values),colors=colors)
ax2.set_title('Churn Channel Sales Count')

plt.show()

Nearly 10% of the client left the company in 2015. There is a change in price scale between churn and non-churn clients. Most of the churned customers are from the same sales channel and origin, so a detailed analysis of that particular part can give more information on factors affecting customer churn rate. Recent customers are more likely to leave the company, The possible reasons can be low competitor's prices or poor customer care.

## Dimensionality Reduction
Before getting into the modeling part, visualizing the orientation of the data can give insights into selecting suitable model for training. Since there were many variables we can use dimensionality reduction to visualize the data.

In [ ]:
df =  pd.get_dummies(df,drop_first=True)
x,y = df.drop('churn',axis=1),df.churn

In [ ]:
pca = PCA(n_components=2)
pca_df = pd.DataFrame(pca.fit_transform(StandardScaler().fit_transform(x)),columns=['PCA1','PCA2'])
pca_df['churn'] = df['churn']
pca_df.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
sns.scatterplot(data=pca_df[pca_df['churn']==1],x='PCA1',y='PCA2')
plt.title('Chrun Plot')
plt.subplot(132)
sns.scatterplot(data=pca_df[pca_df['churn']==0],x='PCA1',y='PCA2')
plt.title('Non Chrun Plot')
plt.subplot(133)
sns.scatterplot(data=pca_df,x='PCA1',y='PCA2',hue='churn')
plt.title('PCA plot')
plt.show()

There some extreme points that makes the plot skewed, Let's remove those points to view more clearly

In [ ]:
lb1,ub1 = iqr_range(pca_df['PCA1'])
pca_df.drop(pca_df[(pca_df['PCA1']<lb1) | (pca_df['PCA1']>ub1)].index,inplace=True)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
sns.scatterplot(data=pca_df[pca_df['churn']==1],x='PCA1',y='PCA2')
plt.title('Chrun Plot')
plt.subplot(132)
sns.scatterplot(data=pca_df[pca_df['churn']==0],x='PCA1',y='PCA2')
plt.title('Non Chrun Plot')
plt.subplot(133)
sns.scatterplot(data=pca_df,x='PCA1',y='PCA2',hue='churn')
plt.title('PCA plot')
plt.show()

*By seeing the PCA plot, we can see that both churn and non-churn customers were not well separated. In the case of predictive modeling, KNN doesn't help much. We didn't see any clear separation boundary or cluster. But there may be a separation in a higher dimension that can be addressed by a hyperplane. If there is no effective hyperplane exists, we can try Tree and Ensemble Based Algorithms. Before getting our hands on predictive modeling, we need to clear the imbalances in the dataset. Let's use the decision tree to choose the correct sample strategy.*


In [ ]:
#Decision Tree without treating the imbalanceness
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.27,stratify=y,random_state=42)
dt_model = tree.DecisionTreeClassifier()
dt_model.fit(x_train,y_train)
y_train_pred = dt_model.predict(x_train)
y_pred = dt_model.predict(x_test)

cm_pred2 = confusion_matrix(y_test,y_pred,labels = dt_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm_pred2,display_labels=dt_model.classes_).plot()
plt.show()

print("Model's f1 score for training dataset :",f1_score(y_train,y_train_pred),
      "\nModel's f1 score for test dataset :",f1_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

## Handling Imbalanceness of the data
**Determining Sampling Strategies for over-sampling the minority class**

In [ ]:
#Balancing the dataset by using simple oversample follow by undersampling
oversample = RandomOverSampler(sampling_strategy=0.5)
undersample = RandomUnderSampler(sampling_strategy='majority')
x_s,y_s = oversample.fit_resample(x,y)
x_s,y_s = undersample.fit_resample(x_s,y_s)
x_train,x_test,y_train,y_test = train_test_split(x_s,y_s,test_size=0.27,stratify=y_s,random_state=42)

In [ ]:
dt_model = tree.DecisionTreeClassifier()
dt_model.fit(x_train,y_train)

y_train_pred = dt_model.predict(x_train)
y_pred = dt_model.predict(x_test)

cm_pred2 = confusion_matrix(y_test,y_pred,labels = dt_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm_pred2,display_labels=dt_model.classes_).plot()
plt.show()

print("Model's f1 score for training dataset :",f1_score(y_train,y_train_pred),
      "\nModel's f1 score for test dataset :",f1_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
#Feature importance as per the model with combined oversampling and undersampling
pd.Series(dt_model.feature_importances_,x_s.columns).sort_values(ascending=False)

In [ ]:
# Balancing dataset using Synthetic Minority Over-sampling Technique with Edited Nearest Neighbour
smt = SMOTEENN(random_state=42,sampling_strategy=0.7)
x_s,y_s = smt.fit_resample(x,y)
x_train,x_test,y_train,y_test = train_test_split(x_s,y_s,test_size=0.27,stratify=y_s,random_state=42)

In [ ]:
dt_model = tree.DecisionTreeClassifier()
dt_model.fit(x_train,y_train)
y_train_pred = dt_model.predict(x_train)
y_pred = dt_model.predict(x_test)

cm_pred2 = confusion_matrix(y_test,y_pred,labels = dt_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm_pred2,display_labels=dt_model.classes_).plot()
plt.show()

print("Model's f1 score for training dataset :",f1_score(y_train,y_train_pred),
      "\nModel's f1 score for test dataset :",f1_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
#Feature importance as per the model with SMOTEENN
pd.Series(dt_model.feature_importances_,x_s.columns).sort_values(ascending=False)

In [ ]:
x_s.hist(figsize=(20,15))
plt.show()

*The synthesized samples by SMOTEENN had a significant impact on feature importance. A discussion with domain expert is needed to verify such values and distributions in the features is possible or not. For this analysis, we use combined undersampling and oversampling as our sampling technique.*
## Combined Over-Sampling and Under-Sampling

In [ ]:
oversample = RandomOverSampler(sampling_strategy=0.5)
undersample = RandomUnderSampler(sampling_strategy='majority')
x_s,y_s = oversample.fit_resample(x,y)
x_s,y_s = undersample.fit_resample(x_s,y_s)
x_train,x_test,y_train,y_test = train_test_split(x_s,y_s,test_size=0.27,stratify=y_s,random_state=42)

##  Model Creation
### Logistic Regression
In the EDA part itself, we have seen that the data were highly skewed, so before applying logistic regression necessary feature transformation is needed for better performance.

In [ ]:
#Distribution of features before transformation
x.hist(figsize=(15,15),bins=10)
plt.show()

In [ ]:
x_scaled = pd.DataFrame(MinMaxScaler().fit_transform(x),columns=x.columns)
q_trans = QuantileTransformer(output_distribution="normal")
x_scaled['cons_12m'] = q_trans.fit_transform(x_scaled['cons_12m'].values.reshape(-1, 1))
x_scaled['net_margin'] = q_trans.fit_transform(x_scaled['net_margin'].values.reshape(-1, 1))
x_scaled['pow_max'] = q_trans.fit_transform(x_scaled['pow_max'].values.reshape(-1, 1))
x_scaled['contract_modification_interval'] = q_trans.fit_transform(x_scaled['contract_modification_interval'].values.reshape(-1, 1))
x_scaled['contract_interval'] = q_trans.fit_transform(x_scaled['contract_interval'].values.reshape(-1, 1))
x_scaled['price_off_peak_fix'] = q_trans.fit_transform(x_scaled['price_off_peak_fix'].values.reshape(-1, 1))
x_scaled['price_mid_peak_fix'] = q_trans.fit_transform(x_scaled['price_mid_peak_fix'].values.reshape(-1, 1))
x_scaled['energy_off_peak_variation'] = q_trans.fit_transform(x_scaled['energy_off_peak_variation'].values.reshape(-1, 1))
x_scaled['cons_gas_12m'] = q_trans.fit_transform(x_scaled['cons_gas_12m'].values.reshape(-1, 1))
x_scaled['imp_cons'] = q_trans.fit_transform(x_scaled['cons_gas_12m'].values.reshape(-1, 1))

In [ ]:
#Distribution of features after transformation
x_scaled.hist(figsize=(15,15),bins=10)
plt.show()

In [ ]:
log_reg = LogisticRegression(random_state=0)
log_reg.fit(x_train,y_train)
yt_pred = log_reg.predict(x_train)
y_pred = log_reg.predict(x_test)
cm_pred2 = confusion_matrix(y_test,y_pred,labels = dt_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm_pred2,display_labels=dt_model.classes_).plot()
plt.show()

print("Model's f1 score for training dataset :",f1_score(y_train,yt_pred),
      "\nModel's f1 score for test dataset :",f1_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

### Support Vector Classification

In [ ]:
svc_model = SVC(random_state=0)
svc_model.fit(x_train,y_train)
yt_pred = svc_model.predict(x_train)
y_pred = svc_model.predict(x_test)
cm_pred2 = confusion_matrix(y_test,y_pred,labels = svc_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm_pred2,display_labels=svc_model.classes_).plot()
plt.show()

print("Model's f1 score for training dataset :",f1_score(y_train,yt_pred),
      "\nModel's f1 score for test dataset :",f1_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

From the results of Logistic Regression and Support Vector Classifier, we can conclude that there is no hyperplane that effectively separate the churn and unchurned customers. Let's try with Decision Tree and Ramdom Forest.

### Decision Tree

In [ ]:
dt_model = tree.DecisionTreeClassifier()
dt_model.fit(x_train,y_train)
y_train_pred = dt_model.predict(x_train)
y_pred = dt_model.predict(x_test)

cm_pred2 = confusion_matrix(y_test,y_pred,labels = dt_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm_pred2,display_labels=dt_model.classes_).plot()
plt.show()

print("Model's f1 score for training dataset :",f1_score(y_train,y_train_pred),
      "\nModel's f1 score for test dataset :",f1_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

### Random Forest

In [ ]:
rf_model = RandomForestClassifier(random_state=0)
rf_model.fit(x_train,y_train)
yt_pred = rf_model.predict(x_train)
y_pred = rf_model.predict(x_test)
cm_pred2 = confusion_matrix(y_test,y_pred,labels = rf_model.classes_)
ConfusionMatrixDisplay(confusion_matrix=cm_pred2,display_labels=rf_model.classes_).plot()
plt.show()

print("Model's f1 score for training dataset :",f1_score(y_train,yt_pred),
      "\nModel's f1 score for test dataset :",f1_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

**From the above classifiers, we can say that the random forest model did the best job of separating churn and non-churn customers. The selection of sampling technique plays a major role here, because the price variation has variable impact based on sampling strategy. But as per the model energy consumption, profit margin, subscribed power, etc has significant importance in the classification task. So we need to investigate the price technique, competitor's subscription plans and discount to arrive precise conclusion.**